# Accessing databases with Python using pyodbc

Before starting, make sure you have installed the `pyodbc` and `pandas` packages in your current environment.
e.g.
```
conda install pandas, pyodbc
```

The [Github repo](https://github.com/mkleehammer/pyodbc) for the `pyodbc` package has a very nice wiki, which includes an excellent [getting started tutorial](https://github.com/mkleehammer/pyodbc/wiki/Getting-started).

In [1]:
import pandas as pd
import pyodbc

# see # https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-SQL-Server-from-Windows
connection_string = "DRIVER={SQL Server}; SERVER=localhost\sqlexpress; DATABASE=Sandbox; Trusted_Connection=yes;"

## Connecting to the database

In [2]:
# Method 1 - Don't use this

conn = pyodbc.connect(connection_string)
# ... do work
conn.close()

# Method 2 - Don't use this

try:
  conn = pyodbc.connect(connection_string)
  # ... do work
finally:
  conn.close()

# Method 3 - USE THIS

with pyodbc.connect(connection_string) as conn:
  # A cursor is a temporary work area created by system memory used to store and manipulate data
  cursor = conn.cursor()
  cursor.execute("SELECT [ItemName], [Color], [Size] FROM [Shirts]")
  row = cursor.fetchone()
  # avoid danger - row may be `None` (which would crash if you try to access its columns )
  if (row):
    print("Full row:", row)
    print("0th column:", row[0])
    print("'Itemname' column:", row.ItemName)

Full row: ('Shirt 1', 'Red', 'Large')
0th column: Shirt 1
'Itemname' column: Shirt 1


## How to iterate across database records

In [3]:
# Method 1 - infinite loop with a break (don't use this)

with pyodbc.connect(connection_string) as conn:
  cursor = conn.cursor()
  cursor.execute("SELECT * FROM [Shirts]")
  while True:
    row = cursor.fetchone()
    if not row:
      break
    print(row)
print("\n")
    
# Method 2 - use iterator

with pyodbc.connect(connection_string) as conn:
  cursor = conn.cursor()
  for row in cursor.execute("SELECT * FROM [Shirts]"):
    print(row)

('Shirt 1', 'Red', 'Large', 10)
('Shirt 2', 'Blue', 'Small', 20)
('Shirt 3', 'Red', 'Small', 15)


('Shirt 1', 'Red', 'Large', 10)
('Shirt 2', 'Blue', 'Small', 20)
('Shirt 3', 'Red', 'Small', 15)


## Filtering your queries, and how to use sql parameters

In [4]:
# Inline strings - Don't use (vulnerable to sql injection attacks, not optimized)
with pyodbc.connect(connection_string) as conn:
  cursor = conn.cursor()
  for row in cursor.execute("SELECT * FROM [Shirts] WHERE [Color]='Red' AND [Cost]>10"):
    print(row)
print("\n")

# Parameterized
with pyodbc.connect(connection_string) as conn:
  cursor = conn.cursor()
  for row in cursor.execute("SELECT * FROM [Shirts] WHERE [Color]=? AND [Cost]>?", "Red", 10):
    print(row)
print("\n")

# Parameterized with "tuple unpacking"
sql = "SELECT * FROM [Shirts] WHERE [Color]=? AND [Cost]>?"
my_params = ("Red", 10)
with pyodbc.connect(connection_string) as conn:
  cursor = conn.cursor()
  for row in cursor.execute(sql, *my_params):
    print(row)

('Shirt 3', 'Red', 'Small', 15)


('Shirt 3', 'Red', 'Small', 15)


('Shirt 3', 'Red', 'Small', 15)


## Make a reusable function for printing the table

In [5]:
def show_shirts(optional_message=None):
  print("\n")
  if (optional_message):
    print(optional_message)    
  with pyodbc.connect(connection_string) as conn:
    cursor = conn.cursor()
    for row in cursor.execute("SELECT * FROM [Shirts]"):
      print(row)
      
# Does it work?
show_shirts()
show_shirts("test")



('Shirt 1', 'Red', 'Large', 10)
('Shirt 2', 'Blue', 'Small', 20)
('Shirt 3', 'Red', 'Small', 15)


test
('Shirt 1', 'Red', 'Large', 10)
('Shirt 2', 'Blue', 'Small', 20)
('Shirt 3', 'Red', 'Small', 15)


## Inserting data

In [6]:
with pyodbc.connect(connection_string) as conn:
  cursor = conn.cursor()
  sql = "INSERT INTO [Shirts] ([ItemName], [Color], [Size], [Cost]) VALUES (?,?,?,?)"
  cursor.execute(sql, ("Shirt X", "Green", "Medium", 30))
show_shirts("inserted 'Shirt X'")



inserted 'Shirt X'
('Shirt 1', 'Red', 'Large', 10)
('Shirt 2', 'Blue', 'Small', 20)
('Shirt 3', 'Red', 'Small', 15)
('Shirt X', 'Green', 'Medium', 30)


## Updating data

In [7]:
with pyodbc.connect(connection_string) as conn:
  cursor = conn.cursor()
  cursor.execute("UPDATE [Shirts] SET [Cost]=? WHERE [ItemName]=?", 25, "Shirt X")
show_shirts("updated 'Shirt X'")



updated 'Shirt X'
('Shirt 1', 'Red', 'Large', 10)
('Shirt 2', 'Blue', 'Small', 20)
('Shirt 3', 'Red', 'Small', 15)
('Shirt X', 'Green', 'Medium', 25)


## Deleting data

In [8]:
with pyodbc.connect(connection_string) as conn:
  cursor = conn.cursor()
  cursor.execute("DELETE FROM [Shirts] WHERE [ItemName]=?", "Shirt X")
show_shirts("deleted 'Shirt X'")



deleted 'Shirt X'
('Shirt 1', 'Red', 'Large', 10)
('Shirt 2', 'Blue', 'Small', 20)
('Shirt 3', 'Red', 'Small', 15)


## Reading data from database directly into a pandas DataFrame

In [9]:
df = {} # <-- want this to persist after the `with` block is out of scope
with pyodbc.connect(connection_string) as conn:
  df = pd.read_sql("SELECT * FROM [Shirts]", conn)
print("\n", df.head())


   ItemName Color   Size  Cost
0  Shirt 1   Red  Large    10
1  Shirt 2  Blue  Small    20
2  Shirt 3   Red  Small    15


In [10]:
# Same thing, but this time as a function.  less akward and more reusable

def get_df():
  with pyodbc.connect(connection_string) as conn:
    return pd.read_sql("SELECT * FROM [Shirts]", conn)

df = get_df()
print("\n", df.head())


   ItemName Color   Size  Cost
0  Shirt 1   Red  Large    10
1  Shirt 2  Blue  Small    20
2  Shirt 3   Red  Small    15
